# Pinecone Canopy library quick start notebook

**Canopy** is a Sofware Development Kit (SDK) for AI applications. Canopy allows you to test, build and package Retrieval Augmented Applications with Pinecone Vector Database. 

This notebook introduce the quick start steps for working with Canopy library. You can find more details about this project and advanced use in the project [documentaion](../README.md).


## Prerequisites

install canopy library

In [1]:
!pip install -qU git+ssh://git@github.com/pinecone-io/canopy.git@dev


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


By default, Canopy uses Pinecone and OpenAI so we need to configure the related API keys.

To get Pinecone free trial API key and environment register or log into your Pinecone account in the [console](https://app.pinecone.io/). You can access your API key from the "API Keys" section in the sidebar of your dashboard, and find the environment name next to it.

You can find your free trial OpenAI API key [here](https://platform.openai.com/account/api-keys). You might need to login or register to OpenAI services.



In [181]:
import os

os.environ["PINECONE_API_KEY"] = os.environ.get('PINECONE_API_KEY') or 'YOUR_PINECONE_API_KEY'
os.environ["PINECONE_ENVIRONMENT"] = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'

## Pinecone Documentation Dataset

Now we'll load a crawl of from 25/10/23 of pinecone docs [website](https://docs.pinecone.io/docs/).

We will use this data to demonstrate how to build a RAG pipepline to answer questions about Pinecone DB.

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


Each record in this dataset represents a single page in Pinecone's documentation. Each row contatins a unique id, the raw text of the page in markdown language, the url of the page as "source" and some metadata. 

## Init a Tokenizer


Many of Canopy's components are using tokenization, which is a process that splits text into tokens - basic units of text (like word or sub-words) that are used for processing. Therefore, Canopy uses a singleton `Tokenizer` object which needs to be initialized once.

In [4]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

After initilizing the global object, we can simply create an instance from anywhere in our code, without providing any parameters:

In [5]:
from canopy.tokenizer import Tokenizer

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

['Hello', ' world', '!']

## Creating a KnowledgBase to store our data for search

`KnowledgeBase` is an object that is responsible for storing and query data. It holds a connection to a single Pinecone index and provides a simple API to insert, delete and search textual documents.

During an upsert, the KnowledgeBase divides the text into smaller chunks, transforms them into vector embeddings, and then upsert these vectors in the underlying Pinecone index. When querying, it converts the textual input into a vector and excute the queries against the underlying index to retrieve the top-k most closely matched chunks.

Here we create a `KnowledgeBase` with our desired index name: 

In [6]:
from canopy.knowledge_base import KnowledgeBase

INDEX_NAME = "my-index"

kb = KnowledgeBase(index_name=INDEX_NAME)

Now we need to create a new index in Pinecone, if it's not already exist:

In [7]:
kb.create_canopy_index(indexed_fields=["title"])

You can see the index created in Pinecone's [console](https://app.pinecone.io/)

next time we would like to init a knowledge base instance to this index, we can simply call the connect method:

In [8]:
kb = KnowledgeBase(index_name=INDEX_NAME)
kb.connect()

> 💡 Note: a knowledge base must be connected to an index before excuting any operation. You should call `kb.connect()` to connect  an existing index or call `kb.create_canopy_index(INDEX_NANE)` before calling any other method of the KB 

## Upsert data to our KnowledgBase

First, we need to convert our dataset to list of `Document` objects

Each document object can hold id, text, source and metadata:

In [9]:
from canopy.models.data_models import Document

example_docs = [Document(id="1",
                      text="This is text for example",
                      source="https://url.com"),
                Document(id="2",
                        text="this is another text",
                        source="https://another-url.com",
                        metadata={"my-key": "my-value"})]

Luckily the columns in our dataset fits this scehma, so we can use a simple iteration to prepare our data:

In [10]:
documents = [Document(**row) for _, row in data.iterrows()]

Now we are ready to upsert our data, with only a single command:

In [11]:
from tqdm.auto import tqdm

batch_size = 10

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/6 [00:00<?, ?it/s]

Internally, the KnowledgeBase handle for use all the processing needed to load data into Pinecone. It chunks the text to smaller pieces and encode them to vectors (embeddings) that can be then upserted directly to Pinecone. Later in this notebook we'll learn how to tune and costumize this process.

## Query the KnowledgeBase

Now we can query the knowledge base. The KnowledgeBase will use its default parameters like `top_k` to exectute the query:

In [12]:
def print_query_results(results):
    for query_results in results:
        print('query: ' + query_results.query + '\n')
        for document in query_results.documents:
            print('document: ' + document.text.replace("\n", "\\n"))
            print('source: ' + document.source)
            print(f"score: {document.score}\n")

In [13]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity")])

print_query_results(results)

query: p1 pod capacity

document: ### s1 pods\n\n\nThese storage-optimized pods provide large storage capacity and lower overall costs with slightly higher query latencies than p1 pods. They are ideal for very large indexes with moderate or relaxed latency requirements.\n\n\nEach s1 pod has enough capacity for around 5M vectors of 768 dimensions.\n\n\n### p1 pods\n\n\nThese performance-optimized pods provide very low query latencies, but hold fewer vectors per pod than s1 pods. They are ideal for applications with low latency requirements (<100ms).\n\n\nEach p1 pod has enough capacity for around 1M vectors of 768 dimensions.
source: https://docs.pinecone.io/docs/indexes
score: 0.842927933

document: ### p2 pods\n\n\nThe p2 pod type provides greater query throughput with lower latency. For vectors with fewer than 128 dimension and queries where `topK` is less than 50, p2 pods support up to 200 QPS per replica and return queries in less than 10ms. This means that query throughput and lat

We can also use metadata filtering and specify `top_k`:

In [25]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity",
                          metadata_filter={"title": "limits"},
                          top_k=2)])

print_query_results(results)

RuntimeError: KnowledgeBase is not connected to index canopy--my-index, Please call knowledge_base.connect(). 

As you can see above, using the metadata filter we get results only from the "limits" page

## Query the Context Engine

While the `KnowledgeBase` is in charge of excuting a textual queries against the Pinecone index, `ContextEngine` is a higher level component that holds the KnowledgeBase, but have a slightly different API:

1. The context engine can get user questions in natural langague. It then generate a search queries out of it. For example, given the question *"What is the capacity of p1 pods?"*, the ContextEngine would first convert it into the search query *"p1 pod capacity"* and then run it against the KnowledgeBase.
2. The `query` method of context engine support a `max_context_tokens` that can limit the number of tokens used in its results. This capabillity allows the user to better handle tokens budgest and limit in the prompts sending later to the LLM.

In [15]:
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [16]:
import json

result = context_engine.query([Query(text="What is the capacity of p1 pods?", top_k=5)], max_context_tokens=512)

print(result.to_text(indent=2))
print(f"\n# tokens in context returned: {result.num_tokens}")

{
  "query": "What is the capacity of p1 pods?",
  "snippets": [
    {
      "source": "https://docs.pinecone.io/docs/indexes",
      "text": "### s1 pods\n\n\nThese storage-optimized pods provide large storage capacity and lower overall costs with slightly higher query latencies than p1 pods. They are ideal for very large indexes with moderate or relaxed latency requirements.\n\n\nEach s1 pod has enough capacity for around 5M vectors of 768 dimensions.\n\n\n### p1 pods\n\n\nThese performance-optimized pods provide very low query latencies, but hold fewer vectors per pod than s1 pods. They are ideal for applications with low latency requirements (<100ms).\n\n\nEach p1 pod has enough capacity for around 1M vectors of 768 dimensions."
    },
    {
      "source": "https://docs.pinecone.io/docs/indexes",
      "text": "### p2 pods\n\n\nThe p2 pod type provides greater query throughput with lower latency. For vectors with fewer than 128 dimension and queries where `topK` is less than 50, p

As you can see above, we queried the context engine with a question in natural language. Also, even though we set `top_k=5`, context engine retreived only 3 results in order to satisfy the 512 tokens limit

## Knowledgeable chat engine

Now we are ready to start chatting with our data!

Canopy `ChatEngine` supports OpenAI compatible API, only that behind the scenes it uses the context egine to provide knowledgeable answers to the users questions.

In [17]:
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

In [18]:
from canopy.models.data_models import MessageBase

response = chat_engine.chat(messages=[MessageBase(role="user", content="What is the capacity of p1 pods?")], stream=False)

print(response.choices[0].message.content)

Each p1 pod has enough capacity for around 1 million vectors of 768 dimensions. [Source: Official Pinecone Documentation](https://docs.pinecone.io/docs/limits)


> 💡 Note: as opposed to OpenAI API, Canopy by default truncate the chat history to recent messages to avoid excceding the prompt tokens limit. This behaviour can change see chat engine [documentation](https://github.com/pinecone-io/canopy/blob/main/src/canopy/chat_engine/chat_engine.py)

## Costumization Example

Canopy built as a modular library, where each component can fully be costumized by the user.

Before we start, we would like to have a quick overview of the inner components used by the knowledge base:

- **Index**: A Pinecone index that holds the vector representations of the documents.
- **Chunker**: A `Chunker` object that is used to chunk the documents into smaller pieces of text.
- **Encoder**: An `RecordEncoder` object that is used to encode the chunks and queries into vector representations.

In the following example, we show how you can costumize the `Chunker` component used by the knowledge base.

First, we will create a dummy chunker class that simply chunks the text by new lines `\n`.

In [19]:
from typing import List
from canopy.knowledge_base.chunker.base import Chunker
from canopy.knowledge_base.models import KBDocChunk

class NewLineChunker(Chunker):

     def chunk_single_document(self, document: Document) -> List[KBDocChunk]:
        line_chunks = [chunk
                       for chunk in document.text.split("\n")]
        return [KBDocChunk(id=f"{document.id}_{i}",
                           document_id=document.id,
                           text=text_chunk,
                           source=document.source,
                           metadata=document.metadata)
                for i, text_chunk in enumerate(line_chunks)]
    
     async def achunk_single_document(self, document: Document) -> List[KBDocChunk]:
        raise NotImplementedError()

In [20]:
chunker = NewLineChunker()

document = Document(id="id1",
                    text="This is first line\nThis is the second line",
                    source="example",
                    metadata={"title": "newline"})
chunker.chunk_single_document(document)

[KBDocChunk(id='id1_0', text='This is first line', source='example', metadata={'title': 'newline'}, document_id='id1'),
 KBDocChunk(id='id1_1', text='This is the second line', source='example', metadata={'title': 'newline'}, document_id='id1')]

Now we can initialize a new knowledge base to use our new chunker:

In [21]:
kb = KnowledgeBase(index_name=INDEX_NAME,
                   chunker=chunker)
kb.connect()

And upsert our example document:

In [22]:
kb.upsert([document])

In [23]:
results = kb.query([Query(text="second line",
                          metadata_filter={"title": "newline"})])

print_query_results(results)

query: second line



As we can see above, our knowledge base split the document by new line as expected.

Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [24]:
kb.delete_index()